In [1]:
from sklearn.ensemble import RandomForestClassifier  # ✅ 引入随机森林
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix
import numpy as np
import pandas as pd

num_folds = 10

train_confusion_matrices_sum = np.zeros((2, 2), dtype=int)
test_confusion_matrices_sum = np.zeros((2, 2), dtype=int)

metrics_dict = {
    'train_accuracy': [], 'test_accuracy': [],
    'train_precision': [], 'test_precision': [],
    'train_recall': [], 'test_recall': [],
    'train_class0_f1': [], 'test_class0_f1': [],
    'train_class1_f1': [], 'test_class1_f1': [],
    'train_f1': [], 'test_f1': [],
    'train_weighted_f1': [], 'test_weighted_f1': [],
    'train_auc': [], 'test_auc': []
}

for fold in range(1, num_folds + 1):
    print(f"\n=== 正在處理 Fold {fold} ===")

    train_file = f'/Users/cleazhang/Downloads/bank-additional-dataset/kfold_fold_{fold}_train.csv'
    test_file = f'/Users/cleazhang/Downloads/bank-additional-dataset/kfold_fold_{fold}_test.csv'

    df_train = pd.read_csv(train_file)
    df_test = pd.read_csv(test_file)
    df_train.drop(columns=['Unnamed: 0'], errors='ignore', inplace=True)
    df_test.drop(columns=['Unnamed: 0'], errors='ignore', inplace=True)

    df_train_class = df_train['y']
    df_train_features = df_train.drop(columns='y')
    df_test_class = df_test['y']
    df_test_features = df_test.drop(columns='y')

    ### ✅ 使用随机森林
    rf_model = RandomForestClassifier(n_estimators=100, random_state=42, class_weight='balanced')
    rf_model.fit(df_train_features, df_train_class)

    predicted_train = rf_model.predict(df_train_features)
    predicted_test = rf_model.predict(df_test_features)
    predicted_prob_train = rf_model.predict_proba(df_train_features)
    predicted_prob_test = rf_model.predict_proba(df_test_features)

    train_confusion_matrices_sum += confusion_matrix(df_train_class, predicted_train)
    test_confusion_matrices_sum += confusion_matrix(df_test_class, predicted_test)

    # 指标计算
    train_accuracy = rf_model.score(df_train_features, df_train_class)
    test_accuracy = rf_model.score(df_test_features, df_test_class)
    metrics_dict['train_accuracy'].append(train_accuracy)
    metrics_dict['test_accuracy'].append(test_accuracy)

    train_precision = precision_score(df_train_class, predicted_train, average='macro')
    test_precision = precision_score(df_test_class, predicted_test, average='macro')
    train_recall = recall_score(df_train_class, predicted_train, average='macro')
    test_recall = recall_score(df_test_class, predicted_test, average='macro')

    train_f1_per_class = f1_score(df_train_class, predicted_train, average=None)
    test_f1_per_class = f1_score(df_test_class, predicted_test, average=None)
    train_f1 = f1_score(df_train_class, predicted_train, average='macro')
    test_f1 = f1_score(df_test_class, predicted_test, average='macro')
    train_weighted_f1 = f1_score(df_train_class, predicted_train, average='weighted')
    test_weighted_f1 = f1_score(df_test_class, predicted_test, average='weighted')

    metrics_dict['train_precision'].append(train_precision)
    metrics_dict['test_precision'].append(test_precision)
    metrics_dict['train_recall'].append(train_recall)
    metrics_dict['test_recall'].append(test_recall)
    metrics_dict['train_class0_f1'].append(train_f1_per_class[0])
    metrics_dict['test_class0_f1'].append(test_f1_per_class[0])
    metrics_dict['train_class1_f1'].append(train_f1_per_class[1])
    metrics_dict['test_class1_f1'].append(test_f1_per_class[1])
    metrics_dict['train_f1'].append(train_f1)
    metrics_dict['test_f1'].append(test_f1)
    metrics_dict['train_weighted_f1'].append(train_weighted_f1)
    metrics_dict['test_weighted_f1'].append(test_weighted_f1)

    train_auc = roc_auc_score(df_train_class, predicted_prob_train[:, 1])
    test_auc = roc_auc_score(df_test_class, predicted_prob_test[:, 1])
    metrics_dict['train_auc'].append(train_auc)
    metrics_dict['test_auc'].append(test_auc)

# === 输出平均性能指标 ===
metrics_summary = {
    metric: f"{np.mean(values):.4f} ± {np.std(values):.4f}" for metric, values in metrics_dict.items()
}

summary_df = pd.DataFrame({
    'Accuracy': [metrics_summary['train_accuracy'], metrics_summary['test_accuracy']],
    'Precision': [metrics_summary['train_precision'], metrics_summary['test_precision']],
    'Recall': [metrics_summary['train_recall'], metrics_summary['test_recall']],
    'Class 0 F1': [metrics_summary['train_class0_f1'], metrics_summary['test_class0_f1']],
    'Class 1 F1': [metrics_summary['train_class1_f1'], metrics_summary['test_class1_f1']],
    'F1 Score': [metrics_summary['train_f1'], metrics_summary['test_f1']],
    'Weighted F1 Score': [metrics_summary['train_weighted_f1'], metrics_summary['test_weighted_f1']],
    'AUC': [metrics_summary['train_auc'], metrics_summary['test_auc']]
}, index=['Train', 'Test'])

print("\n=== 10 折交叉驗證最終結論 ===")
print(summary_df)

summary_df.to_csv('model_evaluation_summary_rf.csv')
np.savetxt('train_confusion_matrix_sum_rf.csv', train_confusion_matrices_sum, delimiter=',', fmt='%d')
np.savetxt('test_confusion_matrix_sum_rf.csv', test_confusion_matrices_sum, delimiter=',', fmt='%d')



=== 正在處理 Fold 1 ===

=== 正在處理 Fold 2 ===

=== 正在處理 Fold 3 ===

=== 正在處理 Fold 4 ===

=== 正在處理 Fold 5 ===

=== 正在處理 Fold 6 ===

=== 正在處理 Fold 7 ===

=== 正在處理 Fold 8 ===

=== 正在處理 Fold 9 ===

=== 正在處理 Fold 10 ===

=== 10 折交叉驗證最終結論 ===
              Accuracy        Precision           Recall       Class 0 F1  \
Train  0.9995 ± 0.0002  0.9981 ± 0.0006  0.9992 ± 0.0012  0.9997 ± 0.0001   
Test   0.8946 ± 0.0126  0.7308 ± 0.0465  0.5901 ± 0.0187  0.9430 ± 0.0072   

            Class 1 F1         F1 Score Weighted F1 Score              AUC  
Train  0.9975 ± 0.0011  0.9986 ± 0.0006   0.9995 ± 0.0002  0.9999 ± 0.0000  
Test   0.2920 ± 0.0471  0.6175 ± 0.0251   0.8719 ± 0.0166  0.7492 ± 0.0333  
